In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
'''
如果出现浏览器显示webgl问题，在谷歌浏览器中输入chrome://flags,将WebGL选项改为Enabled
'''

C:\Users\Administrator\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#运行次数
max_steps = 1001
#图片数量
image_num = 3000
#文件路径,是反斜线，最后记得加上/
DIR = 'C:/Users/Administrator/gitpro/tensorflow/lesson5/'

#定义会话
sess = tf.Session()

#载入图片，把image_num个数据合并为一行
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)#平均值
        #记录值
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))#标准差
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x_t')
    y = tf.placeholder(tf.float32,[None,10],name='y_t')

#显示图片
with tf.name_scope('input_reshape'):
    #把x:[None,784]变换为28*28的矩阵，-1代表None,1代表黑白，如果是彩色则是3(r'g'b)
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)#10张

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        #在网络运行的过程中，保存权值的变化
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([1,10]),name='b')
        variable_summaries(b)
    with tf.name_scope('Wx_plus_b'):
        L1 = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.sigmoid(L1)
        
with tf.name_scope('loss'):
    '''
    就是因为loss没有求平均值，才导致数据格式不对，没有求平均值是数组，求了平均值是数字
    '''
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
    
with tf.name_scope('train'):
    train = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#注意是否初始化变量了，否则程序不会运行也不会报错
sess.run(tf.global_variables_initializer())
    
with tf.name_scope('accuracy_prediction'):
    with tf.name_scope('correct'):
        correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

#产生metadata文件
if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv')
with open(DIR + 'projector/projector/metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')

#合并所有summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
#保存网络的模型
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR+'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR+'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)


for i in range(max_steps):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_ = sess.run([merged,train],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d'%i)
    projector_writer.add_summary(summary,i)
    
    if i%100==0:
        accu= sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('test')
        print("{0} loop,accuracy:{1}".format(i,accu))

#注意文件地址是否正确
saver.save(sess,DIR+'projector/projector/a_model.ckpt',global_step=max_steps)
projector_writer.close()
sess.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

test
0 loop,accuracy:0.2003999948501587
test
100 loop,accuracy:0.8543000221252441
test
200 loop,accuracy:0.86